In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd

In [2]:
def weighted_correlation(x, y, wx, wy):
    mx = np.sum(wx * x) / np.sum(wx)
    my = np.sum(wy * y) / np.sum(wy)
    sx = np.sum(wx * (x - mx)**2) / np.sum(wx)
    sy = np.sum(wy * (y - my)**2) / np.sum(wy)
    sxy = np.sum(np.sqrt(wx * wy) * (x - mx) * (y - my)) / np.sum(np.sqrt(wx * wy))
    return sxy / np.sqrt(sx * sy)

def weighted_covariance(x, y, wx, wy):
    n = x.shape[0]
    mx = np.sum(wx * x) / np.sum(wx)
    my = np.sum(wy * y) / np.sum(wy)
    sxy = np.sum(np.sqrt(wx * wy) * (x - mx) * (y - my)) / np.sum(np.sqrt(wx * wy)) * n / (n - 1) #unbiased
    return sxy 

def weighted_variance(x, wx):
    n = x.shape[0]
    mx = np.sum(wx * x) / np.sum(wx)
    return np.sum(wx * (x - mx)**2) / np.sum(wx) * n / (n - 1) #unbiased

In [3]:
cf = pd.read_csv('data/coef_stat.csv', index_col=0)

In [5]:
### META REGRESSION ###
from pymare import meta_regression
x, y, wx, wy = cf['lmb'], cf['rho'], 1/cf['lmb_V'], 1/cf['rho_V']
lmb_mres = meta_regression(x, wx)
rho_mres = meta_regression(y, wy)
lm, km = lmb_mres.to_df()['estimate'].item(), rho_mres.to_df()['estimate'].item()
ls = weighted_variance(x, wx)
ks = weighted_variance(y, wy)
sxy = weighted_covariance(x, y, wx, wy)

n = cf.shape[0]
mu = np.array([lm, km])
S = np.array([[ls, sxy], 
              [sxy, ks]])

In [8]:
# t squared test from here https://en.wikipedia.org/wiki/Hotelling%27s_T-squared_distribution

def hotteling_t2_test(xnew, mu, S, n, alpha=0.05):
    from scipy.stats import chi2
    p = mu.shape[0]
    d = (xnew - mu) @ np.linalg.inv(S) @ (xnew - mu) #squared mahalanobis distance
    pval = 1 - chi2.cdf(d, p)
    verdict = 'Outlier!' if pval < alpha else 'Plausible!'
    return d, pval, verdict


In [17]:
for i, row in cf.iterrows():
    xnew = np.array([row['lmb'], row['rho']])
    print(i, hotteling_t2_test(xnew, mu, S, n, alpha=0.01))

0 (0.351231913607348, 0.8389401107193468, 'Plausible!')
1 (3.4564261088404957, 0.17760149078133525, 'Plausible!')
2 (6.625824294659117, 0.036409987965231205, 'Plausible!')
3 (4.9941903577092175, 0.08232378751522318, 'Plausible!')
4 (0.8793011199947507, 0.6442615124974969, 'Plausible!')
5 (1.513656399055142, 0.46915212652970706, 'Plausible!')
6 (19.389214077059602, 6.161488737066456e-05, 'Outlier!')
7 (1.2902493845247631, 0.5245971248258301, 'Plausible!')
8 (2.2963821633983477, 0.31721005778551703, 'Plausible!')
9 (3.2770040752108294, 0.1942708348433222, 'Plausible!')
